In [1]:
#!/bin/python3

In [2]:
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = torch.device("cpu")
device

'cuda'

In [4]:
with open("input.txt","r",encoding="utf-8")  as f:
    texts = f.read()

In [5]:
print("here is the length of the no of texts :- ",len(texts))

here is the length of the no of texts :-  1115394


In [6]:
print(texts[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [7]:
#now making characters out of those list
chars = sorted(list(set(texts)))
vocab_size = len(chars)
n_embed =32 # its for nn not for biagram so dont get confused 
print("".join(chars),vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 65


In [8]:
int_string = {s:i for s,i in enumerate(chars)}
string_int = {i:s for s,i in enumerate(chars)}
encoder = lambda x:[ string_int[s] for s in x]# encoder 
decoder = lambda x: "".join([int_string[s] for s in x]) # and decoder 

In [9]:
# so there is process of encoding and decoding the vocabs the current encode process we're using is the general 65 vocab length generic character in the dataset
# but usually GPT2 use Byte pair encoding tokanisation which is bit has more vocab size like 50000 characters insted of 65 character 
#the traid off b/w both of them is that which the no of character in a sentence is large the 64 vocab size length of sequence is huge 
#but in byte paired endcoding tokeniser the same thing uses less #here is the example


#with generic encoding decoding 
some_character= "oye bsdky some thing something something"
print(encoder(some_character))
# with byte paired endcoding tokeniser
import tiktoken
enc = tiktoken.get_encoding("gpt2")
print("\n")
print(enc.encode(some_character))
# see the sequence in tiktoken is not too long as the generic way of doing it 

[53, 63, 43, 1, 40, 57, 42, 49, 63, 1, 57, 53, 51, 43, 1, 58, 46, 47, 52, 45, 1, 57, 53, 51, 43, 58, 46, 47, 52, 45, 1, 57, 53, 51, 43, 58, 46, 47, 52, 45]


[726, 68, 275, 21282, 2584, 617, 1517, 1223, 1223]


In [10]:
data =  torch.tensor(encoder(texts),dtype=torch.long)
data.shape,data.dtype,data[:100]

(torch.Size([1115394]),
 torch.int64,
 tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
         53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
          1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
         57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
          6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
         58, 47, 64, 43, 52, 10,  0, 37, 53, 59]))

In [11]:
# now spliting the data
n = int(0.9*len(data))
train_data = data[:n]
validation_data = data[n:]

In [12]:
# now in this we'll use another approach just like the n-gras but till 8 or any number you wish 
#here is the example
batch_size = 12
block_size =8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for i in range(block_size):
    context = x[:i+1]
    target = y[i]
    print(f"when input is {context} : {target}")
    

when input is tensor([18]) : 47
when input is tensor([18, 47]) : 56
when input is tensor([18, 47, 56]) : 57
when input is tensor([18, 47, 56, 57]) : 58
when input is tensor([18, 47, 56, 57, 58]) : 1
when input is tensor([18, 47, 56, 57, 58,  1]) : 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) : 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) : 58


In [13]:
def get_batch(split):
    data = train_data if split=="train" else validation_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:block_size+i] for i in ix])
    y = torch.stack([data[i+1:block_size+i+1] for i in ix])
    x,y =x.to(device),y.to(device)
    return x,y

In [14]:
#Making Normal biagram
class biagram(nn.Module):
    def __init__(self):
        super().__init__()
        #making an embedding lookup table
        self.token_embedding = nn.Embedding(vocab_size,vocab_size)
    def forward(self,x,y=None):
        logits = self.token_embedding(x)
        if y is  None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            y= y.view(B*T)
            #calculating loss
            loss = F.cross_entropy(logits,y)
        
        return logits,loss 
    def generate(self,x,tokens):
        for i in range(tokens):
            logits,loss = self(x)            
            logits = logits[:,-1,:]
            probs = F.softmax(logits,dim=1)
            next_x = torch.multinomial(probs,num_samples=1)
            x = torch.cat((x,next_x),dim=1)
            
        return x
xb,yb = get_batch("train")
xb,yb =xb.to(device),yb.to(device)
b1 = biagram().to(device=device)
logits,loss = b1(xb,yb)

ix = torch.zeros((1,1),dtype=torch.long).to(device)
prediction = decoder(b1.generate(ix,tokens=100)[0].tolist())
prediction


"\nShlnlIlIgrQgLjpHcJM$Xuo-rOIxfKqotiDvB;sHiGUp$ACGvXlJbMkJRL'tbziRqdv,Z vtL!moNxHggnt\nm?s,Tq&,-QnBhhMA"

In [15]:
optimizer = torch.optim.AdamW(b1.parameters(),lr=1e-2)

for i in range(4000):
 
    xb,yb =get_batch("train")
    # evaluatiing the loss
    optimizer.zero_grad() #turning the grads to zero 
    logits,loss = b1(xb,yb)
    loss.backward()
    optimizer.step() #updating the values 
print(loss.item())
prediction = decoder(b1.generate(ix,tokens=100)[0].tolist())
print(prediction )

2.5641186237335205

TENGLAnowomur amot herou omowect?
ILow;
KEGEOFon tw INSelck ple ld LI budoupat Win
my Fot rt INT:--o


In [16]:
class NN_method(nn.Module):
    def __init__(self):
        super().__init__()
        #each token directly reads off the the logits from the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size,n_embed)# vocab_size.n_embed is global so thats where we are fetching it from
        #we're also taking its possition embeding so herse is how to do it 
        self.position_embeding_table = nn.Embedding(block_size,n_embed)
        #creating a nn 
        self.lm_head = nn.Linear(n_embed,vocab_size)

    def forward(self,x,y=None):
        B,T =x.shape
        token_embedding = self.token_embedding_table(x)
        #using the postion embedding 
        position_emb = self.position_embeding_table(torch.arange(T,device=device))
        x = token_embedding+position_emb
    
        logits = self.lm_head(x)
        if y is None:
            loss = None
        else:
            #scaling it into 2d from  3d
            B,T,C =logits.shape
            logits = logits.view(B*T,C)
            y = y.view(B*T)
            loss = F.cross_entropy(logits,y)
        return logits,loss
    def generate(self,x,max_new_token):
        # x is (B,T) array of indices  in the current context
        for _ in range(max_new_token):
            ix = x[:,-block_size:]
            #get the prediction
            logits,loss =self(ix)
            #focus on the last time stamp
            logits = logits[:,-1,:] #becomes(B,C)
            #apply softmax to get probibilties
            probs = F.softmax(logits,dim=1)
            #sample from the distribution
            x_next = torch.multinomial(probs,num_samples=1) #(B,1)
            #append sample index to running sequence 
            x =torch.cat((x,x_next),dim=1) #(B,T+1)
        return x
model = NN_method().to(device)




In [17]:
ix = torch.zeros((1,1),dtype=torch.long).to(device)
pred = model.generate(x=ix,max_new_token=10)
prediction = decoder(pred[0].tolist())
prediction


'\n$Gjrn\nqU??'

In [18]:
#LOSS evaluation class 
class Loss_evaluation:
    def __init__(self,model,epocs:int):
        self.epocs =epocs
        self.eval_iter =300
        self.model =model
    @torch.no_grad()
    def estimate_loss(self):
        out = {}
        self.model.eval()
        for split in ["train","val"]:
            losses = torch.zeros(self.eval_iter)
            for k in range(self.eval_iter):
                X,Y = get_batch(split)
                logits,loss = self.model(X,Y)
                losses[k] = loss.item()
            out[split]= losses.mean()
        self.model.train()
        return out
    def loss(self):
        for i in range(self.epocs):
            if i%self.eval_iter==0:
                losses=self.estimate_loss()
                print(f"step {i} : train {losses['train']}, test : {losses['val']}")
                #sample a batch of data 
            xb,yb =get_batch("train")
    
            # evaluatiing the loss
            logits,loss = self.model(xb,yb)
            optimizer.zero_grad(set_to_none=True) #turning the grads to zero 
            loss.backward()
            optimizer.step() #updating the values 
        print(loss.item())

In [19]:
optimizer = torch.optim.AdamW(model.parameters(),lr=1e-3)
Loss_evaluation(model,1000).loss()

step 0 : train 4.415390491485596, test : 4.404302597045898
step 300 : train 2.992036819458008, test : 2.993666410446167
step 600 : train 2.754615545272827, test : 2.772991418838501
step 900 : train 2.6750941276550293, test : 2.681922197341919
2.717660665512085


In [20]:
# now generating the words

ix = torch.zeros((1,1),dtype=torch.long,device=device)
print(decoder(model.generate(ix,max_new_token=200)[0].tolist()))



X an
B, ileryou.
Sthalomitoutheedrilyof:
Tke, he fr'ear per:
Ss I japps!sh
HjcBBowseru:
Neo!ef, ce sy IOvve
OO

mtds imat, mye d, t  ghllyeth.
Fe li,
e:  chayoro bs ovucotenMty s'sin serTpfo.
Py t, bo


## Mathmatical trick for self-attention

In [21]:
# now we're gonna use self-attention 
# so how it works is this :- 1. we are totally dependent on the previous outcomes like at [BXTXC] where TXC is dependent on T-1XC OR can say T-2 OR T-0XC 
# and then we find average all previous words  of it and find the prediction

# here's and axample how we do it 

B,T,C =4,8,5
some_x =torch.randn((B,T,C))
x_bag_of_words = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        x_prev = some_x[b,:t+1]
        x_bag_of_words[b,t] = torch.mean(x_prev,0)
x_bag_of_words.shape,x_bag_of_words[0] 

(torch.Size([4, 8, 5]),
 tensor([[ 1.0078,  1.5814,  1.4993,  0.8910, -1.0526],
         [ 0.7135,  0.1649,  0.2068,  0.6118, -1.3212],
         [-0.4626,  0.1551, -0.2281,  0.4731, -1.1227],
         [-0.1096,  0.0230, -0.9178,  0.4050, -1.2176],
         [-0.2847, -0.1277, -0.6658,  0.7190, -1.1999],
         [ 0.0058,  0.1042, -0.8282,  0.9320, -0.8676],
         [ 0.1251,  0.1211, -1.2125,  0.7020, -0.5038],
         [ 0.2407, -0.0394, -1.1678,  0.5832, -0.5018]]))

## anther way of doing it using matrix multiplication

In [22]:
# but there is another way of doing it using matrix multiplication with is very much efficient then doing the cell before trick 
#so here's an explanation for that :- 
a = torch.randint(1,9,(3,4)).float()
b = torch.tril(torch.ones(3,3))
print(f"a   =")
print(a)
print(f"b   =")
print(b)
print(f"dot product  =\n")
print(b@a)
print("\n")
# we did the sim part of the average by using the matrix multiplication but how to do the (sum_of_column)/no_of_column
#lets do the no of columns part and there is a trick to do that here it is :-
b =b/b.sum(1,keepdims=True)
print(f"average  =")
print(b@a)

a   =
tensor([[8., 2., 4., 2.],
        [2., 5., 7., 6.],
        [5., 2., 4., 5.]])
b   =
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
dot product  =

tensor([[ 8.,  2.,  4.,  2.],
        [10.,  7., 11.,  8.],
        [15.,  9., 15., 13.]])


average  =
tensor([[8.0000, 2.0000, 4.0000, 2.0000],
        [5.0000, 3.5000, 5.5000, 4.0000],
        [5.0000, 3.0000, 5.0000, 4.3333]])


In [23]:
#now lets check if it same as the matematical trick we did previously 
wei = torch.tril(torch.ones(8,8))
wei = wei/wei.sum(1,keepdim=True)
x_bag_of_words2 = wei@some_x
torch.allclose(x_bag_of_words2,x_bag_of_words)

True

## there is another way using softmax

In [24]:
#so what we do is use softmax to find the trig veritcal partition of ones here's what i mean
tril = torch.tril(torch.ones(8,8))
wei = torch.zeros((8,8))
wei = wei.masked_fill(tril==0,float("-inf"))
# now making the distribution of 1 in this
wei = F.softmax(wei,dim=1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [25]:
# now we can do the obvious what we did in matrix multiplication
x_bag_of_words3 = wei@some_x
torch.allclose(x_bag_of_words,x_bag_of_words3)

True

##        ----------------------------------- Assembeling every thing ------------------------------

In [26]:
#initializing the variables
batch_size = 14
block_size =100
n_embed = 64
num_head=4
n_layer = 5
dropout_percentage =0.2

In [27]:
def get_batch(split):
    data = train_data if split=="train" else validation_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:block_size+i] for i in ix])
    y = torch.stack([data[i+1:block_size+i+1] for i in ix])
    x,y =x.to(device),y.to(device)
    return x,y

In [28]:
# SO lets make the attention block
class Head(nn.Module):
    #making single head of self attention 
    def __init__(self,head_size):
        super().__init__()
        self.key = nn.Linear(n_embed,head_size,bias=False)
        self.query= nn.Linear(n_embed,head_size,bias=False)
        self.value  = nn.Linear(n_embed,head_size,bias=False)
        self.register_buffer("tril",torch.tril(torch.ones(block_size,block_size)))
        self.dropout = nn.Dropout(dropout_percentage)
    def forward(self,x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = k @ q.transpose(-2,-1) * C**-0.5 
        wei = wei.masked_fill(self.tril[:T, :T]==0,float("-inf"))
        wei =  F.softmax(wei,dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out =  wei @ v
        return out


In [29]:
#Making Multihead attention
class Multihead(nn.Module):
    def __init__(self,No_of_Heads,head_size):
        super().__init__()
        self.heads =nn.ModuleList([Head(head_size) for _ in range(No_of_Heads)])
        self.projection = nn.Linear(n_embed,n_embed)
        self.dropout = nn.Dropout(dropout_percentage)
    def forward(self,x):
        out = torch.cat([i(x) for i in self.heads],dim=-1)
        out = self.dropout(self.projection(out))
        return out 

In [30]:
# making a feed forward layer
class Fedd_forward(nn.Module):
    def __init__(self,dimension):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dimension,4*dimension),
            nn.ReLU(),
            nn.Linear(4*dimension,dimension),
            nn.Dropout(dropout_percentage),
        )
    def forward(self,x):
        return self.net(x)

In [31]:
#Creating a block of all those attention and feed forward layer 
class Block(nn.Module):
    def __init__(self,num_head,n_embed):
        super().__init__()
        self.sa_head = Multihead(No_of_Heads=num_head,head_size=n_embed//num_head)
        self.ffn = Fedd_forward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
        
    def forward(self,x):
        x=x+ self.sa_head(self.ln1(x)) #We did the addition cause of the resedual connection
        x =x+ self.ffn(self.ln2(x))
        return x

In [40]:
class NN_method(nn.Module):
    def __init__(self):
        super().__init__()
        #each token directly reads off the the logits from the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size,n_embed)# vocab_size.n_embed is global so thats where we are fetching it from
        #we're also taking its possition embeding so herse is how to do it 
        self.position_embeding_table = nn.Embedding(block_size,n_embed)
        #creating a block of attention and FFN
        self.block = nn.Sequential(*[Block(num_head,n_embed) for _ in range(n_layer)])
        #layer norm
        self.ln_f = nn.LayerNorm(n_embed)
        #creating a nn 
        self.lm_head = nn.Linear(n_embed,vocab_size)

    def forward(self,x,y=None):
        B,T =x.shape
        token_embedding = self.token_embedding_table(x)
        #using the postion embedding 
        position_emb = self.position_embeding_table(torch.arange(T,device=device))
        x = token_embedding+position_emb
        x= self.block(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        if y==None:
            loss = None
        else:
            #scaling it into 2d from  3d
            B,T,C =logits.shape
            logits = logits.view(B*T,C)
            y = y.view(B*T)
            loss = F.cross_entropy(logits,y)
        return logits,loss
    def generate(self,x,max_new_token):
        # x is (B,T) array of indices  in the current context
        for _ in range(max_new_token):
            ix = x[:,-block_size:]
            #get the prediction
            logits,loss =self(ix)
            #focus on the last time stamp
            logits = logits[:,-1,:] #becomes(B,C)
            #apply softmax to get probibilties
            probs = F.softmax(logits,dim=-1)
            #sample from the distribution
            x_next = torch.multinomial(probs,num_samples=1) #(B,1)
            #append sample index to running sequence
            x =torch.cat((x,x_next),dim=1) #(B,T+1)

        return x        
        
model1 = NN_method().to(device)

In [41]:
optimizer = torch.optim.AdamW(model1.parameters(),lr=1e-2)
Loss_evaluation(epocs=1000,model=model1).loss()

step 0 : train 4.319794654846191, test : 4.320187568664551
step 300 : train 2.402920722961426, test : 2.4247000217437744
step 600 : train 2.1075551509857178, test : 2.1550838947296143
step 900 : train 1.9683036804199219, test : 2.056759834289551
2.0003509521484375


In [42]:
total_params = sum(p.numel() for p in model1.parameters())
print(f"The model has {total_params:,} trainable parameters.")

The model has 263,873 trainable parameters.


In [43]:
context = torch.zeros((1, 1), dtype=torch. long, device=device)
print (decoder(model1.generate (context, max_new_token=500)[0].tolist()))


Noth faucte there,
A dat for grhave dave enould freispen fontir he mangs man athow to krow,
Vore in morre hen rand fite ev hafk,
When mich chich hatt rase conyour.
A wordestress compects, to more
The come fith stis bent mauthece withen the thees wit!

Sightn.

MALIAwIUSCUS:
Ans it and Stry, I bemm, feak, my Jus they geat gize care.

PANGEL:
A'n atitt of bot tue kin shelorcer, sinnot Crueemselys; I arthy canlody sonour'stres.
Ay meln's tor slard? My of I
Your thend or ithall s
Luid Wascomicein; d


In [44]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce GTX 1650


In [45]:
torch.save(model1, 'model1_full.pth') #Saving the model

In [46]:
model1new = torch.load('model1_full.pth')
model1new.to(device)
model1new.eval()

/tmp/ipykernel_15600/2226836182.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model1new = torch.load('model1_full.pth')


NN_method(
  (token_embedding_table): Embedding(65, 64)
  (position_embeding_table): Embedding(100, 64)
  (block): Sequential(
    (0): Block(
      (sa_head): Multihead(
        (heads): ModuleList(
          (0-3): 4 x Head(
            (key): Linear(in_features=64, out_features=16, bias=False)
            (query): Linear(in_features=64, out_features=16, bias=False)
            (value): Linear(in_features=64, out_features=16, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (projection): Linear(in_features=64, out_features=64, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffn): Fedd_forward(
        (net): Sequential(
          (0): Linear(in_features=64, out_features=256, bias=True)
          (1): ReLU()
          (2): Linear(in_features=256, out_features=64, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2):

In [47]:
context = torch.zeros((1, block_size), dtype=torch. long, device=device)
print (decoder(model1.generate (context, max_new_token=2000)[0].tolist()[block_size:]))

CAOUCKICAVENCUSCARIIUS:
O uSTIUS:
LO, are sor,
I'll bed hie leell hou; thoulds and turmffen,
Not ie thish eet me riepnattrome
He not a to Pated ther she morloo's pad.
stacclrlibast, elf tib, matterew Jy chall a procome wast!

Nakentent srer tand sow oughs mond so:
I out oure heefar and forcannouldelmse Cor'd sicincily
I word, not vong terelay your was be, A't
brurk of all shim hiage he mell.
Fortt a thavle he sriself Ere dount ands.
Clist your
That it you a hine nots a That fortiory,
My thould prot and sit gre therake are,
The idmencime that thy 's if reave your muct crendy the
ch bant rut is arthougist chatud ther had sicraved thy
of an it provems, a gatung of to thater
To dombse asfy, shle anisg the brappumbexulin-for'stornt outh of
Te womair lik ingle, somby spiare a Warl themof or edore:
A done mike do firath bar, unhe we to has.

AUSWARICK:
Shou past thee wom go lordo modell, therer my sponoust thus.
Such hemes, then is. That is his fe of-thout n ontrup low thy;
Ay move lorss ulve